### 1) Сравните метрики hit_rate@k, precision@k. Какую матрику использовать предпочтительно и почему. Привидите пример 2-3 задач (опишите, что является клиентом, что товаром), в которой более уместно использовать метрику hit_rate?

Предпочтительней использовать метрику precision@k, поскольку она дает нам точное представление о доле релевантных рекомендаций, в то время как hit_rate@k просто показывает наличие или отсутствие таковых.

<b> Примеры, где можно использовать hit_rate <br> </b>
Пример 1. Онлайн-кинотеатр <br>
Пользователь - зритель, товар - фильм. Когда пользователь заходит в сервис, нам нужно удержать его еще на один фильм, здесь и сейчас. <br>
Пример 2. Предложение однотипных дополнительных товаров к исходной покупке. <br>
Пользователь - покупка, товар - товар. Если человек покупает игровую мышку, то мы рекомендуем ему несколько игровых ковриков, и нас утроит что он купит только один. <br>
Пример 3. Любые магазины, ориентированные на подарочные товары (игрушки, ювелирка и т.д.) <br>
Пользователь - покупатель, товар - товар. Задача рекомендательной системы в том, чтобы помочь клиенту выбрать подарок и не отпустить его в более удобный магазин.

### 2)В метрике NDCG@k мы используем логарифм в знаменателе. Как Вы думаете, почему именно логарифм? Какую функцию можно использовать вместо логарифма? Привидите пример метрик/подходов к предобработке данных/функций ошибок в ML, где также в знаменателе присутствует логарифм.

Мы используем логарфим, потому что его функция возрастает более плавно и со временем скорость возрастания падает (в отличии от y=x). Таким образом, мы сдерживаем рост знаменателя. Можем использовать $$y= \sqrt x$$ <br>
На вторую часть вопроса затрудняюсь ответить, не смог бегло нагуглить

### 3)Какие еще матрики (Вы можете вспомнить уже пройденные Вами или посмотреть в интернете) могут использоваться для рекомендательных систем (приведите примеры метрики и чем являются интеракции, чтобы она могла быть использована)

Из метрик, не освященных на уроке, можно выделить Serendipity like(Diversity, Novelty, Unexpectedness) <br>
(источник: https://towardsdatascience.com/serendipity-accuracys-unpopular-best-friend-in-recommender-systems-ca079b493f3c) <br>
применяются для того, что пользователю разнообразить рекомендации для пользователя и представить ассортимент товаров более широко.


Остальные метрики можно услово разделить на следующие категории: <br>
1) Метрики основанные на матрице ошибок (confusion matrix) для бинарной оценки (понравилась пользователю рекомендация или нет <br>
2) Непрерывные метрики (MSE MAE и т.д.) <br>
3) Метрики ранжирования, если нам важен порядок рекомендаций.

### 4) Посчитайте на этих данных pr@8, rec@8, AP@8, NDCG@8, RR@8, ERR@8

In [1]:
import numpy as np
boughted = [1, 3, 5, 7, 9, 11]
recommended = [2, 5, 7, 4, 11, 9, 8, 10, 12, 3]

#### pr@8

In [2]:
def pr_at_k(boughted, recommended, k): #pr@k
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(boughted_list, recommended_list)
    
    pr = flags.sum() / len(recommended_list)
    
    return pr
    

In [3]:
pr_at_k(boughted, recommended, k=8)

0.5

#### rec@8

In [4]:
def rec_at_k(boughted, recommended, k): #rec@k
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(boughted_list, recommended_list)
    rec = flags.sum() / len(boughted_list)
    
    return rec

In [5]:
rec_at_k(boughted, recommended, k=8)

0.6666666666666666

#### AP@8

In [6]:
def ap_at_k(boughted, recommended, k): #ap@k
    boughted_list = boughted
    recommended_list = recommended
    flags = np.isin(recommended_list, boughted_list)
    
    if sum(flags) == 0:
        return 0
    
    k = sum(flags)
    sum_ = 0
    
    for i in range(1, k+1):
        if flags[i] == True:
            sum_ += pr_at_k(boughted_list, recommended_list, k=i)
    
    result = sum_ / sum(flags)
    
    return result
            

In [7]:
ap_at_k(boughted, recommended, k=8)

0.32

#### NDCG@8

In [8]:
def dcg(flags):
    result = flags[0]/1
    k = len(flags)
    
    for i in range(1, k):
        result += flags[i]/np.log(i+1)
    
    return result/k

In [9]:
def ndcg_a_k(boughted, recommended, k): 
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(recommended_list, boughted_list)
    len_ = len(recommended_list)
    
    dcg_ = dcg(flags)
    idcg = dcg(np.ones(len_))
    
    return dcg_ / idcg
    
    

In [10]:
ndcg_a_k(boughted, recommended, k=8)

0.5653142737255068

#### RR@8

In [11]:
def rec_r(boughted, rec_item):
    rank = 0
    for i, item_bought in enumerate(boughted):
        if rec_item == item_bought:
            return 1/(i+1)
            #rank = 1/(i+1)
    return rank

def mrr_at_k(boughted, recommended, k): 
    boughted_list = boughted
    recommended_list = recommended[:k]
    
    global_rank = 0
    for item in recommended_list:
        global_rank += rec_r(boughted_list, item)
    
    return global_rank / len(recommended_list)

In [12]:
mrr_at_k(boughted, recommended, k=8)

0.11875

#### ERR@8

$$ERR@K = \sum\limits_{k=1}^K \frac{1}{k} P(\textrm{объект остановится на элемента с рангом $k$})$$ <br>
$$P(\textrm{объект остановится на элементе с рангом $k$}) = p_k \prod\limits_{i=1}^{k-1}(1-p_i),$$ <br>

<img src="hw1.png">

In [109]:
def err_at_k(boughted, recommended, k):
    boughted_list = boughted
    recommended_list = recommended[:k]
    flags = np.isin(recommended_list, boughted_list)
    
# высчитывать по общей формуле оказалось е нужно, так как наш r_true либо 0, либо 1. 
#     pk = np.zeros(k)
#     for i in range(k):
#         pk[i] = (2**flags[i] - 1) / (2**max(flags) - 1)
    
    Pk_ = np.zeros(k)
    #Pk_[0] = flags[0]
    #print(flags.astype(int))
    for i in range(0, k):
        prod = np.prod(1 - flags[0:i]) 
        print(i, prod, flags[0:i].astype(int))
        Pk_[i] = flags[i] * prod / (i+1)
        
    print(Pk_)        

    
    return sum(Pk_)
    

In [114]:
print('ERR@8 = ',err_at_k(boughted, recommended, k=8))

[0 1 1 0 1 1 0 0]
0 1 []
1 1 [0]
2 0 [0 1]
3 0 [0 1 1]
4 0 [0 1 1 0]
5 0 [0 1 1 0 1]
6 0 [0 1 1 0 1 1]
7 0 [0 1 1 0 1 1 0]
[0.  0.5 0.  0.  0.  0.  0.  0. ]
ERR@8 =  0.5


Правильно ли я понял алгоритм расчета для примера [0 1 1 ]? <br>

$$1) rank=\frac{1}{1}, p_1 = 0, \prod\limits_{i=1}^{0}(1-p_i)$$ <br>
$$2) rank=\frac{1}{2}, p_2 = 1, \prod\limits_{i=1}^{1}(1-p_i) = 1.$$ <br>
$$3) rank=\frac{1}{3}, p_3 = 1, \prod\limits_{i=1}^{2}(1-p_i) = (1-0) * (1-1) = 0.$$ <br>
Итого по формуле получаем сумму (0 + 1/2 + 0) = 0.5. <br>
Если так, то, наверное, метрика больше подходит для случаев, где r_true может принимать значения не только 1 или 0